In [ ]:
# 安裝必要套件
!pip install langgraph openai


In [ ]:
# 導入模組
from langgraph.graph import StateGraph, END
from dataclasses import dataclass
from typing import Dict
from openai import OpenAI
from google.colab import userdata

# 定義狀態結構
@dataclass
class WorkflowState:
    input: str
    eng: str = ""
    jpn: str = ""
    kor: str = ""
    task: str = "eng"

# 初始化OpenAI客戶端
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))  # 替換為您的API金鑰

# 語言模型調用函數
def llm_call(prompt: str, max_tokens: int = 100) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

# 監督節點：分配翻譯任務
def supervisor_node(state: WorkflowState) -> Dict:
    return {"task": state.task}

# 英文翻譯節點
def eng_node(state: WorkflowState) -> Dict:
    prompt = f"將以下中文翻譯成英文：\n{state.input}"
    state.eng = llm_call(prompt)
    state.task = "jpn"
    return {"eng": state.eng, "task": state.task}

# 日語翻譯節點
def jpn_node(state: WorkflowState) -> Dict:
    prompt = f"將以下中文翻譯成日語：\n{state.input}"
    state.jpn = llm_call(prompt)
    state.task = "kor"
    return {"jpn": state.jpn, "task": state.task}

# 韓語翻譯節點
def kor_node(state: WorkflowState) -> Dict:
    prompt = f"將以下中文翻譯成韓語：\n{state.input}"
    state.kor = llm_call(prompt)
    state.task = "end"
    return {"kor": state.kor, "task": state.task}

# 構建工作流程
workflow = StateGraph(WorkflowState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("eng_node", eng_node)
workflow.add_node("jpn_node", jpn_node)
workflow.add_node("kor_node", kor_node)
workflow.set_entry_point("supervisor")
workflow.add_conditional_edges(
    "supervisor",
    lambda state: state.task,
    {"eng": "eng_node", "jpn": "jpn_node", "kor": "kor_node", "end": END}
)
workflow.add_edge("eng_node", "supervisor")
workflow.add_edge("jpn_node", "supervisor")
workflow.add_edge("kor_node", "supervisor")
graph = workflow.compile()

# 執行範例
result = graph.invoke(WorkflowState(input="人工智慧改變未來"))
print(f"英文翻譯：{result['eng']}")
print(f"日語翻譯：{result['jpn']}")
print(f"韓語翻譯：{result['kor']}")